In [17]:
import numpy as np
import pandas as pd
import os
from PIL import Image
import torch
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms 
import torchvision.transforms as T
from torchvision.models import efficientnet_b0

In [2]:
class GermanTraffic(Dataset):
    def __init__(self, root_dir, transform=None, train=True, verify_ratio=0.2):
        self.root_dir = root_dir
        self.transform = transform
        self.categories = os.listdir(root_dir)
        self.category_mapping = {category: int(category) for idx, category in enumerate(self.categories)}
        self.filepaths = self._get_filepaths()

        if train:
            self.filepaths = self._split_data(verify_ratio)[0]
        else:
            self.filepaths = self._split_data(verify_ratio)[1]

    def __len__(self):
        return len(self.filepaths)

    def __getitem__(self, idx):
        filepath, label = self.filepaths[idx]
        image = Image.open(filepath).convert('RGB')

        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def _get_filepaths(self):
        filepaths = []
        for category in self.categories:
            category_folder = os.path.join(self.root_dir, category)
            image_files = os.listdir(category_folder)
            for image_file in image_files:
                image_path = os.path.join(category_folder, image_file)
                filepaths.append((image_path, self.category_mapping[category]))

        return filepaths

    def _split_data(self, verify_ratio):
        filepaths_train, filepaths_verify = train_test_split(self.filepaths, test_size=verify_ratio)
        return filepaths_train, filepaths_verify


In [3]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.CenterCrop((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_ds = GermanTraffic('../input/acc-german-traffic-sign-classification/GTSRB_Challenge/train/',transform=transform)
valid_ds = GermanTraffic('../input/acc-german-traffic-sign-classification/GTSRB_Challenge/train/',transform=transform,train=False)

In [4]:
train_dl = DataLoader(train_ds,batch_size=64)
valid_dl = DataLoader(valid_ds,batch_size=64)

In [19]:
model = efficientnet_b0(num_classes=43)

In [20]:
device = torch.device('cuda:0')
model = model.to(device)

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim


# Define your loss function
criterion = nn.CrossEntropyLoss()

# Define your optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
num_epochs = 10
# Training loop
for epoch in range(num_epochs):
    # Training phase
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for inputs, labels in train_dl:
        optimizer.zero_grad()
        inputs=inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_train_loss = running_loss / len(train_dl)

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    running_val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in valid_dl:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    avg_val_loss = running_val_loss / len(valid_dl)
    val_accuracy = correct / total

    # Print average loss and accuracy for the epoch
    print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f} | Val Loss = {avg_val_loss:.4f} | Val Accuracy = {val_accuracy:.2%}")



Epoch 1: Train Loss = 3.1658 | Val Loss = 2.1780 | Val Accuracy = 40.98%
Epoch 2: Train Loss = 1.5608 | Val Loss = 0.7329 | Val Accuracy = 81.00%
Epoch 3: Train Loss = 0.6597 | Val Loss = 0.7410 | Val Accuracy = 84.86%
Epoch 4: Train Loss = 0.3133 | Val Loss = 0.1631 | Val Accuracy = 97.64%
Epoch 5: Train Loss = 0.1697 | Val Loss = 0.0716 | Val Accuracy = 98.56%
Epoch 6: Train Loss = 0.1146 | Val Loss = 0.0547 | Val Accuracy = 98.83%
Epoch 7: Train Loss = 0.0831 | Val Loss = 0.0179 | Val Accuracy = 99.53%
Epoch 8: Train Loss = 0.0654 | Val Loss = 0.0082 | Val Accuracy = 99.78%
Epoch 10: Train Loss = 0.0400 | Val Loss = 0.0045 | Val Accuracy = 99.92%


In [22]:
save_path = "gts_state_dict.pth"

# Save only the model's state_dict (learned parameters)
torch.save(model.state_dict(), save_path)

In [23]:
df = pd.read_csv('../input/acc-german-traffic-sign-classification/GTSRB_Challenge/sample_submission.csv')

In [24]:
df = df[['ClassId']==43]

,Filename,ClassId
0,1558973073.8358536.ppm,1
1,1558973073.8359206.ppm,1
2,1558973073.8359365.ppm,1
3,1558973073.8359509.ppm,1
4,1558973073.8359642.ppm,1
...,...,...
12625,1558973073.902373.ppm,1
12626,1558973073.902377.ppm,1
12627,1558973073.9023807.ppm,1
12628,1558973073.9023848.ppm,1


In [ ]:
import pandas as pd
import os
from PIL import Image
import torch
import torchvision.transforms as transforms

# Assuming you have a trained model loaded in the 'model' variable

# Set the path to the submission.csv file and the test folder
submission_file = "../input/acc-german-traffic-sign-classification/GTSRB_Challenge/sample_submission.csv"
test_folder = "../input/acc-german-traffic-sign-classification/GTSRB_Challenge/test"

# Function to load and preprocess the image
def load_image(image_path):
    image = Image.open(image_path)
    transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.CenterCrop((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    return transform(image).unsqueeze(0)

# Load the submission.csv file
submission_df = pd.read_csv(submission_file)

# Initialize lists to store the predictions and file names
predictions = []
file_names = []
#trainedModel = resnet.resnet34(num_classes=43)

    # Load the state dictionary into the model
#trainedModel.load_state_dict(torch.load('../working/gts_state_dict.pth'))

# Loop through the submission dataframe
for index, row in submission_df.iterrows():
    file_name = row['Filename']
    image_path = os.path.join(test_folder, file_name)

    # Ensure the image file exists
    if not os.path.exists(image_path):
        print(f"Image not found: {file_name}")
        continue

    # Load and preprocess the image
    image_tensor = load_image(image_path)
    
    image_tensor = image_tensor.to(device)
    # Make a prediction using the trained model
    with torch.no_grad():
        model.eval()  # Set the model to evaluation mode
        output = model(image_tensor)
        _, predicted_class = torch.max(output, 1)
        predictions.append(predicted_class.item())
        file_names.append(file_name)

# Create a new DataFrame to store the predictions
predictions_df = pd.DataFrame({'Filename': file_names, 'ClassId': predictions})

# Save the predictions to a new CSV file
predictions_df.to_csv('submission.csv', index=False)

        

In [12]:
df = pd.read_csv('../working/submission.csv')